In [ ]:
################################################################## GENERAL IDEA ###################################################################
###################################################################################################################################################
############################################################### 1. PRE PROCESSING #################################################################
###################################################################################################################################################
## Create a data frame from csv file, then create two data frames one for each Station types (start, end) for easier data handling. 
## Making sure data are sorted by datetimes, handling missing data (if any), getting rid of data we do not care about, converting certain columns 
## to certain data types if needed.
## Then, create intervals and calculate values for each interval, handle upper/lower interval problems, and finally creating the final time_serie, 
## handling missing values if necessary. 
## All of the above are being conducted inside a function wich returns the final time series. 
## I did this because I wanted to easily create different time series using different intervals for testing purposes.
## At the end of pre processing, we devide the time series into training and testing data. This is also being conducted in a function because
## I wanted to easily create different training/testing data analogies for testing purposes.
###################################################################################################################################################


**PREPROCESSING OF DATA**

In [ ]:
######################### Time series creating function, using a file name and an integer representing the interval in hours ###########################

# Importing pandas library
import pandas as pd

def csvDataToTimeSerie(fileName, timeInterInHours):  
    # Creating a pandas data frame with the raw data of the csv file
    raw_df = pd.read_csv(fileName)

    # Creating separate data frames for start and end stations
    
    raw_start_df = raw_df[['TripId', 'StartTime', 'StartStationId']]
    raw_end_df = raw_df[['TripId', 'EndTime', 'EndStationId']]
    
    # Making sure the data we are going to work on are sorted by timestamp
    
    raw_start_sorted_df = raw_start_df.sort_values(by='StartTime')
    raw_end_sorted_df = raw_end_df.sort_values(by='EndTime')
    
    # Keeping only data that we want in each data frame (deleting entries where station id is not equal to 1)
    # Also resetting the data frames' indexes for better visualization-manipulation of data
    
    condition = raw_start_sorted_df['StartStationId'] == 1
    raw_start_sorted_df = raw_start_sorted_df[condition]
    raw_start_sorted_df = raw_start_sorted_df.reset_index(drop='True')
    
    condition = raw_end_sorted_df['EndStationId'] == 1
    raw_end_sorted_df = raw_end_sorted_df[condition]
    raw_end_sorted_df = raw_end_sorted_df.reset_index(drop='True')
    
    # Checking if there are NaN values in the the raw data frames
    
    start_df_nan_check = raw_start_sorted_df.isna().any()
    end_df_nan_check = raw_end_sorted_df.isna().any()
    # In this project there are non. If there where, we would have to decide how to deal with the empty spots
    
    # Converting Time columns to timestamps so that certain functions can be used on them
    
    raw_start_sorted_df['StartTime'] = pd.to_datetime(raw_start_sorted_df['StartTime'])
    raw_end_sorted_df['EndTime'] = pd.to_datetime(raw_end_sorted_df['EndTime'])

    # Defining the start and end timestamps of the time series
    # First we compare the min and max timestamps from the two data frames
    # and we keep the overall min and max timestamps
    
    start_time_1 = raw_start_sorted_df['StartTime'].min()
    start_time_2 = raw_end_sorted_df['EndTime'].min()
    if (start_time_1 > start_time_2):
        start_time = start_time_2
    else:
        start_time = start_time_1
    
    end_time_1 = raw_start_sorted_df['StartTime'].max()
    end_time_2 = raw_end_sorted_df['EndTime'].max()
    if (end_time_1 > end_time_2):
        end_time = end_time_1
    else:
        end_time = end_time_2
    
    # Then we round the starting and ending hour (to the floor and ceiling hours respectively)
    
    start_time = start_time.floor('h')
    end_time = end_time.ceil('h')
    
    # This if will take action only if the timeInterInHours exactly devides 24 (total hours per day)
    # Not necessary but if the above is the case, then intervals deviding the total period will not split between different days
    
    if 24 % timeInterInHours == 0:
        start_mod = start_time.hour % timeInterInHours
        if start_mod != 0:
            start_time -= pd.Timedelta(hours=start_mod)
        
        end_mod = end_time.hour % timeInterInHours
        if end_mod != 0:
            end_time += pd.Timedelta(hours=end_mod)
    
    # Creating the final timestamp values of the time series using a frequency of timeInterInHours hours
    # and inserting it into a pandas data frame
    # Each timestamp of the time_serie will indicate the demand for the time period [current_timestamp, current_timestamp+Xhours)
    
    interval = str(timeInterInHours) + 'h'
    timestamps = pd.date_range(start=start_time, end=end_time, freq=interval)
    
    # There is a change that the max timestamp created is less than the end_time of our data.
    # If this is the case we add one more interval, so that we will not miss any data when creating the time serie
    
    if (timestamps[-1] < end_time):
        extra_timestamp = timestamps[-1] + pd.Timedelta(hours=timeInterInHours)
        timestamps = timestamps.append(pd.DatetimeIndex([extra_timestamp]))
    
    # Creating the total period for the time serie
    
    pre_time_serie = pd.DataFrame({'Time': timestamps})
    
    # Calculating the bicycles taken and returned in each period,
    # and then calculating the demand in ech period by subtracting the above
    
    pre_time_serie['Taken'] = 0
    pre_time_serie['Returned'] = 0
    
    # Inserting the taken bicycle values
    i = 0
    max_i = raw_start_sorted_df.index.max()
    max_index = pre_time_serie.index.max()
    for index in range(0, max_index):
        count = 0
        cur_time = raw_start_sorted_df.loc[i, 'StartTime']
        low_time = pre_time_serie.loc[index, 'Time']
        high_time = pre_time_serie.loc[index+1, 'Time']
        while (cur_time>=low_time and cur_time<high_time):
            count += 1
            i += 1
            if (i>max_i): break
            cur_time = raw_start_sorted_df.loc[i, 'StartTime']
        pre_time_serie.loc[index, 'Taken'] = count
        if (i>max_i): break
    
    # Inserting the returned bicycle values
    i = 0
    max_i = raw_end_sorted_df.index.max()
    for index in range(0, max_index):
        count = 0
        cur_time = raw_end_sorted_df.loc[i, 'EndTime']
        low_time = pre_time_serie.loc[index, 'Time']
        high_time = pre_time_serie.loc[index+1, 'Time']
        while (cur_time>=low_time and cur_time<high_time):
            count += 1
            i += 1
            if (i>max_i): break
            cur_time = raw_end_sorted_df.loc[i, 'EndTime']
        pre_time_serie.loc[index, 'Returned'] = count
        if (i>max_i): break
    
    # Creating the resulting time serie by deleted columns that are not needed,
    # deleting the last row, which does not represent a period we want
    # and make the timestamps' column the index
    # We do not have to handle missing values etc,
    # because as we can see from the raw data set, data is spread in any kind day/hour
    
    # If we had to deal with missing values, then usually one of three methods is chosen:
    # 1. Fill with front interval's value with time_serie.Demand.fillna(method = 'bfill') a.k.a "back filling"
    # 2. Fill with back interval's value with time_serie.Demand.fillna(method = 'ffill') a.k.a "front filling"
    # 3. Fill with the mean of all time_serie data with time_serie.Demand.fillna(value = time_serie.Demand.mean()) -- NOT good choice for time series
    #    because usually we observe a periodicity in data and data are strongly connected with neighbor data
    
    pre_time_serie['Demand'] = pre_time_serie['Returned'] - pre_time_serie['Taken']
    time_series = pre_time_serie.copy()
    time_series.drop(columns=['Returned'], inplace=True)
    time_series.drop(columns=['Taken'], inplace=True)
    time_series.drop(time_series.index[-1], inplace=True)
    time_series.set_index('Time', inplace=True)

    freq = str(timeInterInHours) + "h"
    time_series = time_series.asfreq(freq)
    return time_series

############################################ Function to split the time series into training and testing data #######################################

def timeSerieSplit(time_serie, training_perc, testing_perc):
    size = int(len(time_serie)*training_perc)
    training_data = time_serie.iloc[:size]
    testing_data = time_serie.iloc[size:]
    return training_data, testing_data

**TIME SERIES CREATION**

In [ ]:
### Creating a time series from the data.csv file's data, with a 2 hour interval and splitting into 70% training, 30% testing data ###

time_serie = csvDataToTimeSerie('data.csv', 2)
ts, ts_test = timeSerieSplit(time_serie, 0.7, 0.3)

**TIME SERIES ANALYSIS START**

**COMPARING OUR TIME SERIES WITH WHITE NOISE**

In [ ]:
# Plotting the time serie through the data frame

import matplotlib.pyplot as plt
import numpy as np

ts.Demand.plot(figsize = (20, 8))
plt.title("Bicycle Demand")
plt.xlabel("Time")
plt.ylabel("Demand")
plt.grid()
plt.show()

# White noise plot, following a normal distribution with mean and std derived from the time series data
white_noise = np.random.normal(loc = ts.Demand.mean(), scale = ts.Demand.std(), size = len(ts))
ts.loc[:,'white_noise'] = white_noise
print(ts)
ts.white_noise.plot(figsize = (20, 8))
plt.title('White Noise TS')
plt.xlabel('Time')
plt.ylabel('White Noise')
plt.grid()
plt.show()

We can see that the bicycle demand time series plot looks a lot like the white noise time series plot.<br>
They surely both do not have any trend, having a mean very close to zero. The std also seems to be constant.<br>
Regarding to seasonality, in white noise we are sure that there is non there. But in bicycle demand, we see that there could or not exist some daily seasonality. There seems to be some kind of periodicity there, but it does not seem strong. 

**FINDING TREND**

It is obvious by the previous graph that no trend exists in the time series

**FINDING SEASONALITY**

From the above graph it seems that there could exist seasonality of 1 day period.
We will check this hypothesis using autocorrelation.
We can also try to find if there is a correlation in every week's data (84 lags), 
    but with the limited data we have, this examination could lead us to misleading conclusions.

**CORRELATION TEST WITH ACF METHOD**

In [ ]:
import statsmodels.graphics.tsaplots as sgt

sgt.plot_acf(ts.Demand, lags = 84, zero = False)
plt.title("ACF Test")
plt.xlabel("Time")
plt.ylabel("Correlation")
plt.grid()
plt.rcParams["figure.figsize"] = (15, 8)
plt.show()

Even though the ACF method does not give strong direct/indirect correlations between present and past values,
we can see a pattern in the graph with a frequency of 1 day (12 lags x 2 hour interval).
This indicates that MAYBE seasonality exists in the time series.

**CORRELATION TEST WITH PACF METHOD**

In [ ]:
sgt.plot_pacf(ts.Demand, lags = 84, zero = False, method = 'ols')
plt.title("PACF Test")
plt.xlabel("Time")
plt.ylabel("Correlation")
plt.grid()
plt.rcParams["figure.figsize"] = (15, 8)
plt.show()

With the use of PACF method we conclude that there is no significant direct correlation between timestamps of any lag size.
We could try though to use AR model with 5,6,8,9,10,12,66,83 lags maybe, because we see SOME correlation there.<br>
Though direct correlation with 66, 83 lags may be misleading, because we do not have a time series big enough to indicate such a thing with certainty
(our data consist of 235 observations).

**DECOMPOSING SEASONALITY** with the "naive" method

In [ ]:
# We will use seasonal decomposition to split the seasonality from the time series

import statsmodels.tsa.seasonal as stse

s_dec_add = stse.seasonal_decompose(ts.Demand)
s_dec_add.plot()
plt.xticks(rotation = 45)
plt.show()

In [ ]:
ts_seasonality = s_dec_add.seasonal
ts_seasonality = ts_seasonality.to_frame()
ts_seasonality = ts_seasonality.rename(columns={'seasonal': 'season_demand'})
ts_no_season = ts.copy()
ts_no_season['Demand'] = ts['Demand'] - ts_seasonality['season_demand']

In [ ]:
ts.Demand.plot()
plt.rcParams["figure.figsize"] = (30, 5)
plt.title("Original Time Series")
plt.show()
ts_seasonality.season_demand.plot()
plt.title("Seasonality in Time Series")
plt.rcParams["figure.figsize"] = (20, 5)
plt.show()
ts_no_season.Demand.plot()
plt.title("Residuals in Time Series (No Seasonality, No Trend)")
plt.rcParams["figure.figsize"] = (20, 5)
plt.show()

**ACF and PACF TEST for the time series desomposed from seasonality with "naive" method**

In [ ]:
sgt.plot_acf(ts_no_season.Demand, lags = 84, zero = False)
plt.title("ACF Test")
plt.xlabel("Time")
plt.ylabel("Correlation")
plt.grid()
plt.rcParams["figure.figsize"] = (15, 8)
plt.show()

sgt.plot_pacf(ts_no_season.Demand, lags = 84, zero = False, method = 'ols')
plt.title("PACF Test")
plt.xlabel("Time")
plt.ylabel("Correlation")
plt.grid()
plt.rcParams["figure.figsize"] = (15, 8)
plt.show()

In the ACF method we can see that the previous periodicity no longer exist.<br>
Also, once again, neither in the ACF nor the PACF do we observe strong correlation between specific lagged timestamps.

**STATIONARITY TEST**

In [ ]:
import statsmodels.tsa.stattools as sts

adf_ts = sts.adfuller(ts.Demand)
adf_ts_no_season = sts.adfuller(ts_no_season.Demand)

print("ADF Statistic for the ts: ", adf_ts[0])
print("p-value for the ts: ", adf_ts[1], "\n")

print("ADF Statistic for the ts decomposed from seasonal data (with naive method): ", adf_ts_no_season[0])
print("p-value for the ts decomposed from seasonal data (with naive method): ", adf_ts_no_season[1], "\n")

print("critical value (for 5%):", adf_ts[4]['5%'])

We observe that the ADF_statistic < critical_value and that the possibility of the both ts and ts_no_season being non-stationary is close to 0.
It seems safe to declare that both are stationary.

Lets try to run the augmented dickey fuller test, without a max limit of 84 lags (1 week).

In [ ]:
adf_ts_no_lag_limit = sts.adfuller(ts.Demand, maxlag=84)
adf_ts_no_season_no_lag_limit = sts.adfuller(ts_no_season.Demand, maxlag=84)

print("ADF Statistic for the ts: ", adf_ts_no_lag_limit[0])
print("p-value for the ts: ", adf_ts_no_lag_limit[1], "\n")

print("ADF Statistic for the ts decomposed from seasonal data (with naive method): ", adf_ts_no_season_no_lag_limit[0])
print("p-value for the ts decomposed from seasonal data (with naive method): ", adf_ts_no_season_no_lag_limit[1], "\n")

print("critical value (for 5%):", adf_ts_no_lag_limit[4]['5%'])

Observing the results, it appears as both series indeed are stationary.
 Also, considering the amount of data we have (235 observations), using more lags would decrease a lot the degrees of freedom in the test and the test could give very misleading results.

**AR MODEL TEST**

We are assuming that there is no seasonality in place, because AR model generally does not perform well with non-stationary data

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

# As mentioned in previous tests, we will try using the Autoregressive model.
# We will try AR(5),AR(6),AR(8),AR(9),AR(10),AR(12),AR(66),AR(83)
# AR(66) and AR(83) will not be used because they most likely lead to overfitting

ts_model_ar5 = AutoReg(ts.Demand, lags=5).fit()
ts_model_ar6 = AutoReg(ts.Demand, lags=6).fit()
ts_model_ar8 = AutoReg(ts.Demand, lags=8).fit()
ts_model_ar9 = AutoReg(ts.Demand, lags=9).fit()
ts_model_ar10 = AutoReg(ts.Demand, lags=10).fit()
ts_model_ar12 = AutoReg(ts.Demand, lags=12).fit()

pred_ts_ar5 = ts_model_ar5.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
pred_ts_ar6 = ts_model_ar6.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
pred_ts_ar8 = ts_model_ar8.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
pred_ts_ar9 = ts_model_ar9.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
pred_ts_ar10 = ts_model_ar10.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
pred_ts_ar12 = ts_model_ar12.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)

plt.rcParams["figure.figsize"] = (15, 8)
plt.subplot(4, 2, 1)
plt.plot(pred_ts_ar5)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("AR(5)")

plt.subplot(4, 2, 2)
plt.plot(pred_ts_ar6)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("AR(6)")

plt.subplot(4, 2, 3)
plt.plot(pred_ts_ar8)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("AR(8)")

plt.subplot(4, 2, 4)
plt.plot(pred_ts_ar9)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("AR(9)")

plt.subplot(4, 2, 5)
plt.plot(pred_ts_ar10)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("AR(10)")

plt.subplot(4, 2, 6)
plt.plot(pred_ts_ar12)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("AR(12)")

plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import root_mean_squared_error as rmse
print('MAE')
print("AR(5):", round(mae(ts_test.Demand, pred_ts_ar5), 3))
print("AR(6):", round(mae(ts_test.Demand, pred_ts_ar6), 3))
print("AR(8):", round(mae(ts_test.Demand, pred_ts_ar8), 3))
print("AR(9):", round(mae(ts_test.Demand, pred_ts_ar9), 3))
print("AR(10):", round(mae(ts_test.Demand, pred_ts_ar10), 3))
print("AR(12):", round(mae(ts_test.Demand, pred_ts_ar12), 3), "\n")

print('RMSE')
print("AR(5)", round(rmse(ts_test.Demand, pred_ts_ar5), 3))
print("AR(6):", round(rmse(ts_test.Demand, pred_ts_ar6), 3))
print("AR(8):", round(rmse(ts_test.Demand, pred_ts_ar8), 3))
print("AR(9):", round(rmse(ts_test.Demand, pred_ts_ar9), 3))
print("AR(10):", round(rmse(ts_test.Demand, pred_ts_ar10), 3))
print("AR(12):", round(rmse(ts_test.Demand, pred_ts_ar12), 3))

Even though the AR model when using a lot of lags it appears to be following the test data a little better, as we can see
 from the calculated errors they are not the best choices.<br>
From the above results our best choices would be AR(10) and AR(12).

After some tests, I found that the AR(45) model is the one that gives the best prediction with minimum MAE, RMSE

In [ ]:
ts_model_ar45 = AutoReg(ts.Demand, lags=45).fit()
pred_ts_ar45= ts_model_ar45.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
plt.plot(pred_ts_ar45)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("AR(45)")
plt.show()
print("MAE AR(45):", round(mae(ts_test.Demand, pred_ts_ar45), 3))
print("RMSE AR(45)", round(rmse(ts_test.Demand, pred_ts_ar45), 3))

In [ ]:
# Checking the residuals/error term for the AR(45) model we created
plt.plot(ts_model_ar45.resid, 'o')
plt.show()
print("Residuals mean: ", round(ts_model_ar45.resid.mean(), 3))
print("Residuals variance: ", round(ts_model_ar45.resid.var(), 3))

We can see that the residuls do have a mean of almost zero, but they are not gathered around zero. They are scattered all
over the place. This is a strong indication that the AR(45) model is performing poorly.
Also, we must have in mind that the more lags we add, the more chance there is for overfitting, and the more we are fitting our model to our specific data.  

**ARMA MODEL TEST**

We are assuming that there is no seasonality in place, because AR model generally does not perform well with non-stationary data

Again we take a look at the ACF and PACF tests for our time series

In [ ]:
import statsmodels.graphics.tsaplots as sgt

sgt.plot_acf(ts.Demand, lags = 84, zero = False)
plt.title("ACF Test")
plt.xlabel("Time")
plt.ylabel("Correlation")
plt.grid()
plt.rcParams["figure.figsize"] = (15, 8)
plt.show()

sgt.plot_pacf(ts.Demand, lags = 84, zero = False, method = 'ols')
plt.title("PACF Test")
plt.xlabel("Time")
plt.ylabel("Correlation")
plt.grid()
plt.rcParams["figure.figsize"] = (15, 8)
plt.show()

**Deciding ARMA model parameters**

Parameter regarding AR part:
We can see from the PACF that at first there is no correlation, then some strong correlation occurs with 4-10 lags, 11 is low and
12 is high again. There are some more lags indicating correlation later on (e.g.  20, 36 etc), but we will ignore them as they will probably lead to overfitting.
So for the AR part we will use the 4, 5, 6, 7, 8, 9, 10 and 12 lags for our tests.

Parameter regarding MA part:
We can see from the ACF that is strong correlation with 4, 5, 10, 11 first lags. Again, there are some more lags indicating correlation later on (e.g.  23, 36 etc), but we will ignore them too.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import root_mean_squared_error as rmse

# AR part parameter range
p_par = range(4, 13, 1)
# MA part parameter range
q_par = (4, 5, 10, 11)
dic = dict()
count = 0

for p in p_par:
    for q in q_par:
        order = (p, 0, q)
        model = ARIMA(ts.Demand, order=order).fit()
        pred_model = model.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
        
        mae_value = round(mae(ts_test.Demand, pred_model), 3)
        rmse_value = round(rmse(ts_test.Demand, pred_model), 3)
        x = "For the ARMA(" + str(p) + "," + str(q) + ") Trained Model's prediction, we have MAE=" + str(mae_value) + " and RMSE=" + str(rmse_value)
        dic[count] = x
        count = count + 1 

In [ ]:
for key in dic:
    print(dic[key])

In [ ]:
# ARMA(6, 11) and ARMA(7, 4) seem to be the best choices with MAE~3.74 and RMSE~5.0

order = (6, 0, 11)
ar6_ma11 = ARIMA(ts.Demand, order=order).fit()
pred_ar6_ma11 = ar6_ma11.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)

order = (7, 0, 4)
ar7_ma4 = ARIMA(ts.Demand, order=order).fit()
pred_ar7_ma4 = ar7_ma4.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)

plt.subplot(2, 1, 1)
plt.plot(pred_ar6_ma11)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARMA(6, 11)")

plt.subplot(2, 1, 2)
plt.plot(pred_ar7_ma4)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARMA(7, 4)")

The above warnings could be generated for various reasons. One of the most common is data being non-stationary.

After ARMA model we will try the ARIMA model, as well as the SARIMA model, which we hope to address the (potential) seasonality issue

**ARIMA MODEL TEST**

In [ ]:
# Trying different integration levels for AR param=6, 7 and MA param=11, 4

dic = dict()

order = (6, 1, 11)
ar6_d1_ma11 = ARIMA(ts.Demand, order=order).fit()
pred_ar6_d1_ma11 = ar6_d1_ma11.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_ar6_d1_ma11)
rmse_value = rmse(ts_test.Demand, pred_ar6_d1_ma11)
dic[0] = "For the ARIMA" + str(order) + " Trained Model's prediction we have MAE=" + str(mae_value) + " and RMSE=" + str(rmse_value)

order = (6, 2, 11)
ar6_d2_ma11 = ARIMA(ts.Demand, order=order).fit()
pred_ar6_d2_ma11 = ar6_d2_ma11.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_ar6_d2_ma11)
rmse_value = rmse(ts_test.Demand, pred_ar6_d2_ma11)
dic[1] = "For the ARIMA" + str(order) + " Trained Model's prediction we have MAE=" + str(mae_value) + " and RMSE=" + str(rmse_value)

order = (7, 1, 4)
ar7_d1_ma4 = ARIMA(ts.Demand, order=order).fit()
pred_ar7_d1_ma4 = ar7_d1_ma4.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_ar7_d1_ma4)
rmse_value = rmse(ts_test.Demand, pred_ar7_d1_ma4)
dic[2] = "For the ARIMA" + str(order) + " Trained Model's prediction we have MAE=" + str(mae_value) + " and RMSE=" + str(rmse_value)

order = (7, 2, 4)
ar7_d2_ma4 = ARIMA(ts.Demand, order=order).fit()
pred_ar7_d2_ma4 = ar7_d2_ma4.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_ar7_d2_ma4)
rmse_value = rmse(ts_test.Demand, pred_ar7_d2_ma4)
dic[3] = "For the ARIMA" + str(order) + " Trained Model's prediction we have MAE=" + str(mae_value) + " and RMSE=" + str(rmse_value)

In [ ]:
for key in dic:
    print(dic[key])

In [ ]:
# Best ARIMA model seems to be the ARIMA(7, 1, 4)

plt.plot(pred_ar7_d1_ma4)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARIMA(7, 1, 4)")
plt.show()

We actually did not expect the ARIMA model to perform better than ARMA model, as ARIMA offers to address trend issues,
and there is no kind of trend to be addressed in our data.

**SARIMA MODEL TEST**

In case seasonality exists and plays important role in our time series, then we hope SARIMA model will take this into account
and will give us better predictions.

In [ ]:
from pmdarima import auto_arima

# Trying to get best for our model with auto_arima function
# m = 12 because the seasonality we expect is daily, which is every 12 observations (1 observation/ 2 hours)
model = auto_arima(y = ts.Demand, m = 12)
print(model)

In [ ]:
print("Best SARIMA model fit found to be ARIMA(0,0,0)(2,0,0)[12]")

sarima_model = ARIMA(ts.Demand, order=(0,0,0), seasonal_order=(2,0,0,12)).fit()
pred_sarima = sarima_model.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_sarima)
rmse_value = rmse(ts_test.Demand, pred_sarima)
print("For the ARIMA(0,0,0)(2,0,0)[12] Model's prediction we have MAE=", str(mae_value), " and RMSE=", str(rmse_value))
plt.plot(pred_sarima)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title(model)
plt.show()

Lets try different SARIMA parameters to see if we will have better results.

In [ ]:
sar10_ima12 = ARIMA(ts.Demand, order=(10,0,12), seasonal_order=(2,0,0,12)).fit()
pred_sar10_ima12 = sar10_ima12.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_sar10_ima12)
rmse_value = rmse(ts_test.Demand, pred_sar10_ima12)
print("For the ARIMA(10,0,12)(2,0,0)[12] Model's prediction we have MAE=", str(mae_value), " and RMSE=", str(rmse_value))

sar6_ima11 = ARIMA(ts.Demand, order=(6,0,11), seasonal_order=(2,0,0,12)).fit()
pred_sar6_ima11 = sar6_ima11.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_sar6_ima11)
rmse_value = rmse(ts_test.Demand, pred_sar6_ima11)
print("For the ARIMA(6,0,11)(2,0,0)[12] Model's prediction we have MAE=", str(mae_value), " and RMSE=", str(rmse_value))

sar7_ima4 = ARIMA(ts.Demand, order=(7,0,4), seasonal_order=(2,0,0,12)).fit()
pred_sar7_ima4 = sar7_ima4.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_sar7_ima4)
rmse_value = rmse(ts_test.Demand, pred_sar7_ima4)
print("For the ARIMA(7,0,4)(2,0,0)[12] Model's prediction we have MAE=", str(mae_value), " and RMSE=", str(rmse_value))

sar10_ima1 = ARIMA(ts.Demand, order=(10,0,1), seasonal_order=(2,0,0,12)).fit()
pred_sar10_ima1 = sar10_ima1.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_sar10_ima1)
rmse_value = rmse(ts_test.Demand, pred_sar10_ima1)
print("For the ARIMA(10,0,1)(2,0,0)[12] Model's prediction we have MAE=", str(mae_value), " and RMSE=", str(rmse_value))

plt.subplot(2, 2, 1)
plt.plot(pred_sar10_ima12)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARIMA(10,0,12)(2,0,0)[12]")

plt.subplot(2, 2, 2)
plt.plot(pred_sar6_ima11)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARIMA(6,0,11)(2,0,0)[12]")

plt.subplot(2, 2, 3)
plt.plot(pred_sar7_ima4)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARIMA(7,0,4)(2,0,0)[12]")

plt.subplot(2, 2, 4)
plt.plot(pred_sar10_ima1)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARIMA(10,0,1)(2,0,0)[12]")

plt.show()

Lets test the ARIMA(7,0,4)(2,0,0)[12], but with different integration levels to see if it will result in better prediciton

In [ ]:
sar7_i1_ma4 = ARIMA(ts.Demand, order=(7,1,4), seasonal_order=(2,0,0,12)).fit()
pred_sar7_i1_ma4 = sar7_i1_ma4.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_sar7_i1_ma4)
rmse_value = rmse(ts_test.Demand, pred_sar7_i1_ma4)
print("For the ARIMA(7,1,4)(2,0,0)[12] Model's prediction we have MAE=", str(mae_value), " and RMSE=", str(rmse_value))

sar7_i2_ma4 = ARIMA(ts.Demand, order=(7,2,4), seasonal_order=(2,0,0,12)).fit()
pred_sar7_i2_ma4 = sar7_i2_ma4.predict(start=len(ts), end=len(time_serie)-1, dynamic=False)
mae_value = mae(ts_test.Demand, pred_sar7_i2_ma4)
rmse_value = rmse(ts_test.Demand, pred_sar7_i2_ma4)
print("For the ARIMA(7,2,4)(2,0,0)[12] Model's prediction we have MAE=", str(mae_value), " and RMSE=", str(rmse_value))

plt.subplot(2, 1, 1)
plt.plot(pred_sar7_i1_ma4)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARIMA(7,2,4)(2,0,0)[12]")

plt.subplot(2, 1, 2)
plt.plot(pred_sar7_i2_ma4)
plt.plot(ts_test.Demand)
plt.xticks(rotation=45)
plt.title("ARIMA(7,2,4)(2,0,0)[12]")

plt.show()